<a href="https://colab.research.google.com/github/kostuk/WarAudioMap/blob/main/SoundMap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install tensorflow

In [1]:
%pip install keras

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install tflite-model-maker


  Using cached tflite_model_maker-0.4.3-py3-none-any.whl (580 kB)
  Using cached tflite_model_maker-0.4.2-py3-none-any.whl (577 kB)
  Using cached tflite_model_maker-0.4.1-py3-none-any.whl (642 kB)
  Using cached tflite_model_maker-0.4.0-py3-none-any.whl (642 kB)
  Using cached tensorflow_hub-0.12.0-py2.py3-none-any.whl (108 kB)
  Using cached librosa-0.8.1-py3-none-any.whl (203 kB)
  Using cached lxml-5.1.0-cp310-cp310-win_amd64.whl (3.9 MB)
  Using cached tflite_model_maker-0.3.4-py3-none-any.whl (616 kB)
  Using cached tflite_model_maker-0.3.3-py3-none-any.whl (616 kB)
  Using cached tflite_model_maker-0.3.2-py3-none-any.whl (591 kB)
  Using cached tensorflow_hub-0.9.0-py2.py3-none-any.whl (103 kB)
  Using cached tflite_model_maker-0.3.1-py3-none-any.whl (590 kB)
  Using cached tflite_model_maker-0.3.0-py3-none-any.whl (567 kB)
  Using cached tflite_model_maker-0.2.5-py3-none-any.whl (499 kB)
  Using cached neural_structured_learning-1.4.0-py2.py3-none-any.whl (128 kB)
  Using cache

ERROR: Exception:
Traceback (most recent call last):
  File "e:\work\soundMap\.vevn\lib\site-packages\pip\_vendor\urllib3\response.py", line 435, in _error_catcher
    yield
  File "e:\work\soundMap\.vevn\lib\site-packages\pip\_vendor\urllib3\response.py", line 516, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "e:\work\soundMap\.vevn\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 94, in read
    self.__buf.write(data)
  File "C:\Users\Nikolas\AppData\Local\Programs\Python\Python310\lib\tempfile.py", line 483, in func_wrapper
    return func(*args, **kwargs)
OSError: [Errno 28] No space left on device

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "e:\work\soundMap\.vevn\lib\site-packages\pip\_internal\cli\base_command.py", line 167, in exc_logging_wrapper
    status = run_func(*args)
  File "e:\work\soundMap\.vevn\lib\site-packages\pip\_internal\cli\req_command.py", line 247, in

In [1]:
import tensorflow as tf
import tflite_model_maker as mm
from tflite_model_maker import audio_classifier
import os
import pathlib

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import glob
import random

from IPython.display import Audio, Image
from scipy.io import wavfile

print(f"TensorFlow Version: {tf.__version__}")
print(f"Model Maker Version: {mm.__version__}")


import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import models


ModuleNotFoundError: No module named 'tflite_model_maker'

In [ ]:
DATASET_PATH = '/content/drive/MyDrive/soundMap'

data_dir = pathlib.Path(DATASET_PATH)

commands = np.array(tf.io.gfile.listdir(str(data_dir)))
commands = commands[ commands != 'none']
print('Commands:', commands)

filenames = tf.io.gfile.glob(str(data_dir) + '/*/*')

    
    
filenames = tf.random.shuffle(filenames)
num_samples = len(filenames)
print('Number of total examples:', num_samples)
print('Number of examples per label:',
      len(tf.io.gfile.listdir(str(data_dir/commands[0]))))
print('Example file tensor:', filenames[0])


train_files = filenames[:110]
val_files = filenames[110: 110 + 40]
test_files = filenames[-40:]

print('Training set size', len(train_files))
print('Validation set size', len(val_files))
print('Test set size', len(test_files))


code_to_name = {
  'pushka': 'Pushka',
  'unknow': 'Unknow'
}





In [ ]:
spec = audio_classifier.YamNetSpec(
    keep_yamnet_and_custom_heads=True,
    frame_step=audio_classifier.YamNetSpec.EXPECTED_WAVEFORM_LENGTH,
    frame_length=audio_classifier.YamNetSpec.EXPECTED_WAVEFORM_LENGTH//2)

In [ ]:
def get_label(file_path):
  parts = tf.strings.split(
      input=file_path,
      sep=os.path.sep)
  # Note: You'll use indexing here instead of tuple unpacking to enable this
  # to work in a TensorFlow graph.
  return parts[-2]

In [ ]:
audio_classifier.YamNetSpec.EXPECTED_WAVEFORM_LENGTH

In [ ]:
train_data = audio_classifier.DataLoader.from_folder(
    spec, data_dir, cache=True)

In [ ]:

train_data, validation_data = train_data.split(0.6)
test_data, validation_data = validation_data.split(0.5)


In [ ]:
print(train_data.size)
print(validation_data.size)
print(test_data.size)

In [ ]:
batch_size = 64
epochs = 100

print('Training the model')
model = audio_classifier.create(
    train_data,
    spec,
    validation_data,
    batch_size=batch_size,
    epochs=epochs)

In [ ]:
print('Evaluating the model')
model.evaluate(test_data)

In [ ]:
def show_confusion_matrix(confusion, test_labels):
  """Compute confusion matrix and normalize."""
  confusion_normalized = confusion.astype("float") / confusion.sum(axis=1)
  axis_labels = test_labels
  ax = sns.heatmap(
      confusion_normalized, xticklabels=axis_labels, yticklabels=axis_labels,
      cmap='Blues', annot=True, fmt='.2f', square=True)
  plt.title("Confusion matrix")
  plt.ylabel("True label")
  plt.xlabel("Predicted label")

confusion_matrix = model.confusion_matrix(test_data)
show_confusion_matrix(confusion_matrix.numpy(), test_data.index_to_label)

In [ ]:
serving_model = model.create_serving_model()

print(f'Model\'s input shape and type: {serving_model.inputs}')
print(f'Model\'s output shape and type: {serving_model.outputs}')

In [ ]:
test_files = os.path.abspath(os.path.join(DATASET_PATH, '*/*.wav'))

def get_random_audio_file():
  test_list = glob.glob(test_files)
  print(test_list)
  random_audio_path = random.choice(test_list)
  return random_audio_path


def show_sound_data(audio_path):
  sample_rate, audio_data = wavfile.read(audio_path, 'rb')

  bird_code = audio_path.split('/')[-2]
  print(f'code: {bird_code}')

  plttitle = f'{bird_code}'
  plt.title(plttitle)
  plt.plot(audio_data)
  display(Audio(audio_data, rate=sample_rate))

print('functions and data structures created')

In [ ]:
# if you want to try another file just uncoment the line below
print(test_files)
random_audio = get_random_audio_file()
print(random_audio)
show_sound_data(random_audio)

In [ ]:
sample_rate, audio_data = wavfile.read(random_audio, 'rb')

audio_data = np.array(audio_data) / tf.int16.max
input_size = serving_model.input_shape[1]

splitted_audio_data = tf.signal.frame(audio_data, input_size, input_size, pad_end=True, pad_value=0)

print(f'Test audio path: {random_audio}')
print(f'Original size of the audio data: {len(audio_data)}')
print(f'Number of windows for inference: {len(splitted_audio_data)}')

In [ ]:
print(random_audio)

results = []
print('Result of the window ith:  your model class -> score,  (spec class -> score)')
for i, data in enumerate(splitted_audio_data):
  yamnet_output, inference = serving_model(data)
  results.append(inference[0].numpy())
  result_index = tf.argmax(inference[0])
  spec_result_index = tf.argmax(yamnet_output[0])
  t = spec._yamnet_labels()[spec_result_index]
  result_str = f'Result of the window {i} {spec_result_index}: ' \
  f'\t{test_data.index_to_label[result_index]} -> {inference[0][result_index].numpy():.3f}, ' \
  f'\t({spec._yamnet_labels()[spec_result_index]} -> {yamnet_output[0][spec_result_index]:.3f})'
  print(result_str)


results_np = np.array(results)
mean_results = results_np.mean(axis=0)
result_index = mean_results.argmax()
print(f'Mean result: {test_data.index_to_label[result_index]} -> {mean_results[result_index]}')

In [ ]:
models_path = './pushki_models'
print(f'Exporing the TFLite model to {models_path}')

model.export(models_path, tflite_filename='pushki_model.tflite')